In [ ]:
# Install required libraries
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=86e059778ae7251482b1e3b623bb5a95639b0da9b221ac2c3933da877bce3cfe
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [ ]:
import os

# Install packages
os.system('pip install textblob')
os.system('pip install scikit-surprise')
os.system('pip install pandas numpy')

# Download TextBlob corpora
from textblob import download_corpora
download_corpora.download_all()


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [ ]:
import pandas as pd
import numpy as np

# Load the product reviews dataset
product_reviews = pd.read_csv('/content/productReviews.csv')
product_list=pd.read_csv('/content/productlist.csv')
# Step 1: Generate random user IDs
# Define number of users you want to simulate
num_users = 1000

# Assign a random user_id to each review, where similar user_id can be repeated for different products
product_reviews['user_id'] = np.random.randint(1, num_users + 1, product_reviews.shape[0])

# Check the dataset with user_id
print(product_reviews.head())
print(product_list.head())

   sno    product_id                                             review  \
0    0  4.669760e+12  This makes my skin smooth and soft and is ligh...   
1    1  4.669760e+12  Love the silky texture. It's very lightweight ...   
2    2  4.669760e+12  I’ve been trying to find a moisturizer that wo...   
3    3  4.669760e+12  HYRAM made me buy it and I’m on my second bott...   
4    4  4.669760e+12  It's a nice moisturizer I personally will use ...   

   user_id  
0      308  
1      920  
2      534  
3      131  
4      107  
   Unnamed: 0    product_id           product_name  product_brand price  \
0           0  6.562640e+12             VITALIFT-A  Dr. Different   $42   
1           1  6.562640e+12       VITALIFT-A Forte  Dr. Different   $52   
2           2  6.562640e+12  VITALIFT-A Eye & Neck  Dr. Different   $40   
3           3  6.592230e+12   Great Barrier Relief    KraveBeauty   $28   
4           4  6.535230e+12    Oasis Soothing Mask      DR ALTHEA   $18   

                    

In [ ]:
from textblob import TextBlob

# Generate ratings from review sentiment
def sentiment_to_rating(review_text):
    # Using TextBlob to perform sentiment analysis
    sentiment = TextBlob(review_text).sentiment.polarity

    # Convert sentiment polarity (-1 to 1) to rating scale (1 to 5)
    rating = round((sentiment + 1) * 2.5)  # Scale sentiment from (-1, 1) to (1, 5)
    return rating

# Apply the sentiment_to_rating function to generate the 'rating' column
product_reviews['rating'] = product_reviews['review'].apply(sentiment_to_rating)

# Check the updated dataset with generated ratings
print(product_reviews[['user_id', 'product_id', 'rating']].head())


   user_id    product_id  rating
0      308  4.669760e+12       3
1      920  4.669760e+12       3
2      534  4.669760e+12       3
3      131  4.669760e+12       4
4      107  4.669760e+12       2


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# Prepare the data for collaborative filtering
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(product_reviews[['user_id', 'product_id', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# Define a user-based collaborative filtering model with Pearson similarity
sim_options = {
    'name': 'pearson_baseline',  # Using Pearson similarity
    'user_based': True  # User-based collaborative filtering
}
user_knn_model = KNNBasic(sim_options=sim_options)

# Train the model
user_knn_model.fit(trainset)

# Test the model
predictions = user_knn_model.test(testset)
accuracy.rmse(predictions)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.6846


0.6846177374537139

In [ ]:

# Recommend products for a specific user
def recommend_products_for_user(user_id, n=5):
    product_ids = product_reviews['product_id'].unique()

    # Predict ratings for all products the user has not rated yet
    predicted_ratings = []
    for product_id in product_ids:
        prediction = user_knn_model.predict(user_id, product_id)
        predicted_ratings.append((product_id, prediction.est))

    # Sort the products by the predicted rating
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    recommended_product_ids = [x[0] for x in predicted_ratings[:n]]

    # Fetch product details from product_list for these recommendations
    recommended_products = product_list[product_list['product_id'].isin(recommended_product_ids)]
    return recommended_products

# Example usage: Recommend products for a specific user
user_id_to_recommend = 400  # Replace with an actual user ID from your dataset
recommendations = recommend_products_for_user(user_id_to_recommend)
print(f"Recommended products for user {user_id_to_recommend}:\n{recommendations}")


Recommended products for user 400:
     Unnamed: 0    product_id  \
88          110  4.480000e+12   
131         170  1.706910e+12   
148         197  4.477080e+12   
151         200  4.422860e+12   
189         248  1.888570e+12   

                                          product_name product_brand   price  \
88                      Charcoal Bubble Cleansing Pads      MEDIHEAL  $11.99   
131               Ultra Vitalizing Snail Essence Water      DEWYTREE     $38   
148                             Fermentation Mask Pack        BENTON      $3   
151                MadeCera Cream Double Essence Toner    SKINRX LAB     $26   
189  SUR.MEDIC+ Super Hyaluronic 100TM Amino Acid C...    SUR.MEDIC+     $20   

                                   product_description product_type  
88   An innovative and simple one-step dual-sided c...     Cleanser  
131  This lightweight yet powerful essence hydrates...      Essence  
148  This premium mask pack from Benton contains th...         Mask  
151  

In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np
from sklearn.metrics import roc_auc_score

# Prepare the data for collaborative filtering
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(product_reviews[['user_id', 'product_id', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# Define a user-based collaborative filtering model with Pearson similarity
sim_options = {
    'name': 'pearson_baseline',  # Using Pearson similarity
    'user_based': True  # User-based collaborative filtering
}
user_knn_model = KNNBasic(sim_options=sim_options)

# Train the model
user_knn_model.fit(trainset)

# Test the model
predictions = user_knn_model.test(testset)
accuracy.rmse(predictions)

# Function to evaluate precision@k and recall@k
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    from collections import defaultdict

    # Get top-N recommendations for each user
    def get_top_n(predictions, n=5):
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in predictions:
            top_n[uid].append((iid, est))
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]
        return top_n

    top_n = get_top_n(predictions, n=k)

    precisions = []
    recalls = []

    # Iterate through testset using predictions instead of unpacking testset
    relevant_items_per_user = defaultdict(list)
    for uid, iid, true_r in testset:
        if true_r >= threshold:
            relevant_items_per_user[uid].append(iid)

    for uid, user_ratings in top_n.items():
        relevant_items = relevant_items_per_user[uid]
        recommended_items = [iid for (iid, _) in user_ratings]

        num_relevant_and_recommended = len(set(recommended_items) & set(relevant_items))
        precision = num_relevant_and_recommended / len(recommended_items) if recommended_items else 0
        recall = num_relevant_and_recommended / len(relevant_items) if relevant_items else 0

        precisions.append(precision)
        recalls.append(recall)

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    return avg_precision, avg_recall


# Function to compute Mean Average Precision (MAP)
def mean_average_precision(predictions):
    from collections import defaultdict
    user_map_scores = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_map_scores[uid].append((iid, true_r, est))

    map_scores = []
    for user_ratings in user_map_scores.values():
        user_ratings.sort(key=lambda x: x[2], reverse=True)  # Sort by predicted score
        num_relevant = 0
        score = 0
        for rank, (iid, true_r, _) in enumerate(user_ratings):
            if true_r >= 3.5:  # Consider threshold as relevant
                num_relevant += 1
                score += num_relevant / (rank + 1)
        if num_relevant > 0:
            map_scores.append(score / num_relevant)

    return np.mean(map_scores)

# Function to compute AUC (Area Under the ROC Curve)
def compute_auc(predictions):
    y_true = [1 if true_r >= 3.5 else 0 for (_, _, true_r, _, _) in predictions]
    y_scores = [est for (_, _, _, est, _) in predictions]
    return roc_auc_score(y_true, y_scores)

# Evaluate metrics
precision, recall = precision_recall_at_k(predictions, k=5)
map_score = mean_average_precision(predictions)
auc_score = compute_auc(predictions)

print(f"Precision@K: {precision}")
print(f"Recall@K: {recall}")
print(f"MAP: {map_score}")
print(f"AUC: {auc_score}")

# Recommend products for a specific user
def recommend_products_for_user(user_id, n=5):
    product_ids = product_reviews['product_id'].unique()

    # Predict ratings for all products the user has not rated yet
    predicted_ratings = []
    for product_id in product_ids:
        prediction = user_knn_model.predict(user_id, product_id)
        predicted_ratings.append((product_id, prediction.est))

    # Sort the products by the predicted rating
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    recommended_product_ids = [x[0] for x in predicted_ratings[:n]]

    # Fetch product details from product_list for these recommendations
    recommended_products = product_list[product_list['product_id'].isin(recommended_product_ids)]
    return recommended_products

# Example usage: Recommend products for a specific user
user_id_to_recommend = 400  # Replace with an actual user ID from your dataset
recommendations = recommend_products_for_user(user_id_to_recommend)
print(f"Recommended products for user {user_id_to_recommend}:\n{recommendations}")


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.6633
Precision@K: 0.20468856947296374
Recall@K: 0.5000855578370978
MAP: 0.60098826975022
AUC: 0.5226616129272081
Recommended products for user 400:
     Unnamed: 0    product_id  \
80          102  4.620650e+12   
81          103  4.620650e+12   
88          110  4.480000e+12   
131         170  1.706910e+12   
148         197  4.477080e+12   
189         248  1.888570e+12   

                                          product_name product_brand   price  \
80                  Houttuynia Cordata Calming Essence        GOODAL     $30   
81           Houttuynia Cordata Calming Moisture Cream        GOODAL     $24   
88                      Charcoal Bubble Cleansing Pads      MEDIHEAL  $11.99   
131               Ultra Vitalizing Snail Essence Water      DEWYTREE     $38   
148                             Fermentation Mask Pack        BENTON      $3   
189  SUR.MEDIC+

In [ ]:
def ndcg_at_k(predictions, k=5, threshold=3.5):
    from collections import defaultdict

    def dcg_at_k(ranked_list, k):
        dcg = 0
        for i, (iid, _) in enumerate(ranked_list[:k]):
            if iid in relevant_items:
                dcg += 1 / np.log2(i + 2)  # Discounted by rank position
        return dcg

    # Get top-N recommendations for each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    # Compute NDCG@K for each user
    ndcg_scores = []
    for uid, user_ratings in top_n.items():
        relevant_items = set([iid for iid, true_r in product_reviews[product_reviews['user_id'] == uid][['product_id', 'rating']].values if true_r >= threshold])
        dcg = dcg_at_k(user_ratings, k)
        ideal_dcg = dcg_at_k(sorted(user_ratings, key=lambda x: x[1], reverse=True), k)
        ndcg_scores.append(dcg / ideal_dcg if ideal_dcg > 0 else 0)

    return np.mean(ndcg_scores)

# Example usage:
ndcg_score = ndcg_at_k(predictions, k=5)
print(f"NDCG@K: {ndcg_score}")


NDCG@K: 0.6129363449691991


In [ ]:
def f1_score_at_k(predictions, k=5, threshold=3.5):
    from collections import defaultdict

    # Get top-N recommendations for each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    # Calculate precision and recall
    precisions = []
    recalls = []
    for uid, user_ratings in top_n.items():
        relevant_items = set([iid for iid, true_r in product_reviews[product_reviews['user_id'] == uid][['product_id', 'rating']].values if true_r >= threshold])
        recommended_items = set([iid for iid, _ in user_ratings[:k]])

        tp = len(relevant_items & recommended_items)
        fp = len(recommended_items) - tp
        fn = len(relevant_items) - tp

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0

        precisions.append(precision)
        recalls.append(recall)

    avg_f1 = np.mean([2 * p * r / (p + r) if (p + r) > 0 else 0 for p, r in zip(precisions, recalls)])

    return avg_f1

# Example usage:
f1_score = f1_score_at_k(predictions, k=5)
print(f"F1-Score@K: {f1_score}")


F1-Score@K: 0.2525509170273031


In [ ]:
def hit_rate_at_k(predictions, k=5, threshold=3.5):
    from collections import defaultdict

    # Get top-N recommendations for each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    # Calculate Hit Rate
    hit_rates = []
    for uid, user_ratings in top_n.items():
        relevant_items = set([iid for iid, true_r in product_reviews[product_reviews['user_id'] == uid][['product_id', 'rating']].values if true_r >= threshold])
        recommended_items = set([iid for iid, _ in user_ratings[:k]])

        hit = 1 if len(relevant_items & recommended_items) > 0 else 0
        hit_rates.append(hit)

    return np.mean(hit_rates)

# Example usage:
hit_rate = hit_rate_at_k(predictions, k=5)
print(f"Hit Rate@K: {hit_rate}")


Hit Rate@K: 0.6129363449691991


In [ ]:
from collections import defaultdict

def coverage_at_k(predictions, k=5):
    recommended_items = set()
    total_items = len(product_list['product_id'].unique())

    # Get top-N recommendations for each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    # Collect all recommended items
    for uid, user_ratings in top_n.items():
        recommended_items.update([iid for iid, _ in user_ratings[:k]])

    return len(recommended_items) / total_items

# Example usage:
coverage_score = coverage_at_k(predictions, k=5)
print(f"Coverage@K: {coverage_score}")



Coverage@K: 0.9694656488549618


In [ ]:
def mrr_at_k(predictions, k=5, threshold=3.5):
    from collections import defaultdict

    # Get top-N recommendations for each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)

    # Calculate MRR@K
    mrr_scores = []
    for uid, user_ratings in top_n.items():
        relevant_items = set([iid for iid, true_r in product_reviews[product_reviews['user_id'] == uid][['product_id', 'rating']].values if true_r >= threshold])
        rank = -1
        for i, (iid, _) in enumerate(user_ratings[:k]):
            if iid in relevant_items:
                rank = i + 1
                break
        mrr_scores.append(1 / rank if rank != -1 else 0)

    return np.mean(mrr_scores)

# Example usage:
mrr_score = mrr_at_k(predictions, k=5)
print(f"MRR@K: {mrr_score}")


MRR@K: 0.4678473648186174
